<a href="https://colab.research.google.com/github/shstreuber/Data-Mining/blob/master/Module8_DeepLearning2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Module 8: Neural Networks and Deep Learning**
Imagine you have lots of data--Big Data, as in 1,000,000 tuples and more per day--and need to build a classification system with utmost reliability because if you're wrong, the consequences may be detrimental to people or property. Would you use a simple tree? Or a k Nearest Neighbor? Or a Random Forest? Or would you want a system that combines a number of self-optimizing algorithm runs with an element of randomization and voting in order to give you the most reliable output?

That, then would be a Deep Learning Network. **Deep Learning means nothing more than a Neural Network with multiple hidden layers,** in which data is summarized and analyzed and summarized and analyzed and so on. These pictures say it all:

**A Simple Neural Network**
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/simplenn_regression.png" width="350">
</div>

**A Deep Neural Network**
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/deepnn_regression.png" width="600">
</div>

At the end of this module, you will be able to:

* Configure 2 simple and 2 deep learning Networks for Regression and Classification
* Describe how a Deep Neural Network works
* Configure TensorFlow and Keras
* Solve a simple Deep Learning problem
* Compare regular Neural Network output with Deep Learning output

To get started, please watch this instructor video:

In [2]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://www.youtube.com/embed/RkiTL_T8VsY", width=560, height=315)

#**What is Tensorflow?**
Essentially, TensorFlow (a Google product) is an end-to-end open source machine learning **platform**. As a platform, it contains a number of libraries, or packages, the most well-known of which is Keras.

The **GOAL** of TensorFlow is to train and run deep neural networks for handwritten digit classification, image recognition, word embeddings, recurrent neural networks, sequence-to-sequence models for machine translation, natural language processing, and PDE (partial differential equation) based simulations.

##**Sooooo ... what is a Tensor?**
The vocabulary here may sound technical and daunting, but there are a few very simple concepts hiding behind it. A Tensor is simply a multidimensional array:

* Scalar = 0D Tensor
* Vector = 1D Tensor
* Matrix = 2D Tensor
* Cube = 3D Tensor

Thereby, we can deduce that a 4-D tensor is a vector of cubes, 5-D tensor is a matrix of cubes, 6-D tensor is a cube of cubes, etc. Take a look at the graphic below:

<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/tensor.png" width="600">
</div>

##**And How does it Work?**
Working with TensorFlow is  basically like setting up any other Classification:
0. You set up your libraries and load your data
1. You do your EDA (Exploratory Data Analysis) to see how the data is distributed and to determine what the class attribute in the dataset should be.
2. Preprocess the data (remove n/a, transform data types as needed, deal with missing data) and THEN normalize the data so we can apply the model weights without problems.
3. Split the data into a training set and a test set
4. Build the model based on the training set
5. Test the model on the test set
6. Determine the quality of the model

#**0. Preparation and Setup**
To wrap our head around the process of setting up a Deep Learning model, we will work with a dataset with which we are already familiar: The adult dataset.

In [ ]:
import tensorflow as tf # This tells Colab that we are using TensorFlow

from tensorflow import keras # This is the main TensorFlow library
from tensorflow.keras import layers # We are building a Neural Network with several hidden layers
from tensorflow.keras.layers.experimental import preprocessing

print("Current TensorFlow version is", tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # for visualization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

#Reading in the data as adult dataframe
adult = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/adult.data.simplified25.csv")
adult.head()

#**1. Exploratory Data Analysis**
How many rows does the adult dataset have? What are the attribute types? What is the mean, median, and mode of the incomeUSD attribute?

These are all questions to solve. Use the code rows below to find the answers:

# **A2. Regression**
As with the simple Neural Networks you have encountered already, preprocessing is a bit more involved than with, say, a Random Forest algorithm.
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/TF_Process1.png" width="600">
</div>

In our first case study, we will be working on Regression, which means we will be predicting ONE NUMBER, i.e. the expected IncomeUSD

##**2.1 Reducing the Data**


##Your Turn
You have done this before with the insurance dataset: Build an adult_dl dataset consisting of age, educationyears, race, hoursperweek, and incomeUSD as the class attribute

##**2.2 Preparing the Data for use with TensorFlow**
In this section, you will see that preparing data to work with a Deep Learning Neural Network requires the same kind of preprocessing that you have already encountered:



### **2.2.1 Encoding Categorical Variables**
Did you remember that Neural Networks (regular **and** in TensorFlow) require only numeric data? Well ... "race" is quite obviously categorical, so we need to convert it to one-hot format. We do this with pd.get_dummies()

In [ ]:
adult_dl = pd.get_dummies(adult_dl, columns=['race'], prefix='', prefix_sep='')
adult_dl.head()

### **2.2.2 Splitting into Training and Test Set**
We will do this first since we will want to reduce the amount of data that we will have to normalize.

In [ ]:
train_dataset = adult_dl.sample(frac=0.8, random_state=0)
test_dataset = adult_dl.drop(train_dataset.index)

### **2.2.3 Splitting Features from Labels**
Separate the target value, the "label", from the features. This label is the value that you will train the model to predict--in our case, we want to predict incomeUSD.

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('incomeUSD')
test_labels = test_features.pop('incomeUSD')

### **2.2.4 Normalizing**
Except for age and educationyears, which are both measured in years, all other variables are measured in **different units**. This also puts them on **different scales**. Since, in a Neural Network, we need all our values on the **same scale**, so that the weights can be applied uniformly, we need to take action! One reason this is important is because the features are multiplied by the model weights. So the scale of the outputs and the scale of the gradients are affected by the scale of the inputs.

In the previous module, you've learned how mean-normalization with StandardScaler() works. This time, we are going to do this differently by setting up an actual layer INSIDE our neural network.

The [**preprocessing.Normalization layer**](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) is a clean and simple way to build this preprocessing into your model. And YES--by creating the normalization layer, you effectively just started building your TensorFlow model:

In [ ]:
normalizer = preprocessing.Normalization(axis=-1) # axis=-1 means that normalization should be applied to the last axis (i.e., each feature independently)

Now we apply the normalizer to the data:

In [ ]:
train_features = train_features.astype(np.float32) # Convert the data type from int to float32 so the normalizer can recognize them easily

normalizer.adapt(train_features) # Adapt the normalizer to the train_features data

normalized_features = normalizer(train_features) # Normalize the train_features

first = np.array(train_features[:1]) # Get the first sample from train_features for demonstration

with np.printoptions(precision=2, suppress=True): # Print the original and normalized data with specified precision
    print('Original data:', first)
    print()
    print('Normalized data:', normalizer(first).numpy())

Above, you can see what happens inside the normalization layer: It returns the input data, with each feature independently normalized.

###**WAIT! Some of the normalized data is negative. Is that correct?**
Yes, it is correct for normalized data to sometimes have negative values. Here's why:

When using the Normalization layer from TensorFlow (or other standardization techniques), the normalization process typically involves centering the data around a mean of zero and scaling it based on the standard deviation. This process is often referred to as Z-score normalization or standardization.

**Explanation**

The formula for standardization is:

> standardized_value = (original_value − mean)/ standard_deviation

* Mean (average): The center of the data.
* Standard Deviation: How spread out the data is from the mean.

By subtracting the mean from each data point and then dividing by the standard deviation, the resulting data will have:

* A mean of 0.
* A standard deviation of 1.

This transformation means that data points that were below the mean in the original dataset will have negative standardized values, and those above the mean will have positive standardized values.
<center>
<img src = "https://static9.depositphotos.com/1007989/1112/i/450/depositphotos_11129291-stock-illustration-nerdy-smiley.jpg" height = 200>
</center>

# **A3. Building the Models**
There is always a specific process with which to build a TensorFlow model:
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/TF_Process2.png" width="600">
</div>

1. First, we set up the **keras SEQUENTIAL MODEL**. This is the framework inside of which we are going to define the layers. Sequential = layers are sequentially next to each other (either “stacked” or left-to-right, depending on how you draw them).
---
2. Inside the Sequential model, we define the **LAYERS**. To do this, we need to know the following:
* **Shape**: This is the number of attributes we use as input for the model
---
3. In the next step, we define HOW we want the model to run, that is to **COMPILE**, with model.compile(). To do this, we need to know the following:
* **Optimizer** = gradient descent function (i.e. which function we use to optimize the step-down of the weights); adam = adaptive learning rate optimization algorithm
* **Loss Function**= evaluation of the ŷ vs the ground truth
* **Metrics** = evaluation criterion, here accuracy.
---
4. Then, we **FIT** the model to the training set with model.fit(). To do this, we need to know the following:
* **Epoch**: One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE. If one epoch is too big to feed to the computer at once we can divide it in several smaller batches
* **Batch size**: Depending on the number of needed features in your dataset (you should reduce these to NO MORE THAN 6), the computing effort can be too intense. Just like you would not each a whole sandwich in one bite, the machine does better when processing the data in smaller bites called batches. The standard batch size is 32.
---
5. Lastly (and in purple!), we use our model to **PREDICT** the values for the test set with model.predict()
---
**How we choose the LOSS FUNCTION** for step 3 depends on the type of calculation we need our Neural Network to perform:
* If the output variable is **continuous**, we are performing a regression, so the loss function is **mean squared error or MSE**
* If the output variable is **binary**, we are performing a classification, so the loss function is **binary_crossentropy**
* If the output variable is **categorical** with more than two labels, we are still performing a classification, but now the loss function is **categorical_crossentropy**



##**3.1 REGRESSION: Using one Variable to predict incomeUSD in a Simple Neural Network**
We will use age to predict incomeUSD. To do so, we will use the typical keras.Sequential model. This model represents a sequence of steps. In this case there are two steps:

1. Normalize the input ['age'].
2. Apply a linear transformation () to produce one output using layers.Dense.

The number of inputs can either be set by the input_shape argument, or automatically when the model is run for the first time.

In [ ]:
# First, we build the normalization layer:
age_new = np.array(train_features['age'])

age_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
age_normalizer.adapt(age_new)

In [ ]:
# Now we build the framework that holds all the models:
age_model = tf.keras.Sequential([
    age_normalizer,
    layers.Dense(units=1)
])

age_model.summary()

Do you see how the Output Shape contains 1? That's only one number in one layer, so a very basic Neural Network.

Now we can configure the training procedure using the Model.compile() method. The most important arguments to compile are the loss and the optimizer since these define what will be optimized (mean_absolute_error) and how (using the [optimizers.Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)).

In [ ]:
age_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

Once the model is configured, we use Model.fit() to train it over 100 epochs (give this about 1-2 minutes):

In [ ]:
%%time
# The %%time magic command measures and shows the time it takes to execute the entire cell. This is helpful for performance monitoring and optimization.
history = age_model.fit(
    train_features['age'], train_labels,
    epochs=100,
    # suppress logging
    verbose=1,
    # Calculate validation results on 20% of the training data. Validation means that we test as we go, on a 20% subset of the training data
    validation_split = 0.2)

The output looks something like this:

```
Epoch 1/100
500/500 [==============================] - 1s 2ms/step - loss: 56370.8867 - val_loss: 55092.4062
Epoch 2/100
500/500 [==============================] - 1s 2ms/step - loss: 55920.8945 - val_loss: 54642.5195
Epoch 3/100
500/500 [==============================] - 1s 2ms/step - loss: 55470.8594 - val_loss: 54192.6094
Epoch 4/100
500/500 [==============================] - 1s 2ms/step - loss: 55020.7891 - val_loss: 53742.7422
```
Here, you are seeing TWO loss outputs: loss and val_loss.

In the output of a machine learning training process, loss and val_loss are key metrics used to monitor the model's performance.

* **loss**: The training loss is the error calculated on the training dataset after each epoch. It shows how well the model is fitting the training data. This loss is minimized during the training process.
* **val_loss**: The validation loss is the error calculated on the validation dataset after each epoch. It provides an estimate of the model's performance on *unseen data* (the validation set), which helps in detecting overfitting. Overfitting occurs when the model performs well on training data but poorly on validation data.

###**But the numbers are really large!**

Yes, they are. When the loss numbers are very large, as seen in our example, it typically indicates that the model's predictions are significantly off from the actual values. Here are some potential reasons and implications of large loss values:

1. **Scale of Target Variable:**
The target variable (output) might be on a very large scale. For example, if you are predicting monetary values that range in the tens of thousands, the loss values can naturally be large if they are not normalized or scaled appropriately.
2. **Initial Model Parameters:** The initial weights of the model might be poorly initialized, leading to poor predictions in the initial epochs.
3. **Learning Rate:** If the learning rate is too high, the model may be making large updates to the weights, which can result in large error values and unstable training.
4. **Model Complexity:** The model might be too simple to capture the complexity of the data, resulting in poor predictions and thus high loss values.
5. **Data Issues:** There could be outliers or errors in the data, which can cause high loss values. For example, extreme values in the target variable can skew the loss.
6. **Inappropriate Loss Function:** Using a loss function that does not suit the problem can lead to high loss values. For example, using mean squared error (MSE) for a classification problem instead of cross-entropy loss.

**HOWEVER**, despite the huge numbers, both loss and val_loss are decreasing, which is a good sign indicating that the model is learning and improving on both the training and validation datasets. Monitoring both loss and val_loss during training helps ensure that our model is learning properly and generalizing well to new data, which is critical for building robust machine learning models.

In [ ]:
train_features['age']

Visualize the model's training results (you must run the code fields to see the graph):

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = age_model.predict(x)

In [ ]:
def plot_age(x, y):
  plt.scatter(train_features['age'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('age')
  plt.ylabel('incomeUSD')
  plt.legend()
plot_age(x,y)

##Your Turn

Look at the scatter plot. What does the data distribution vs. the predictor line tell you? Write your answer in the field below.


Now let's test this on the test set and collect the results so we can inspect them at the end of this file:

In [ ]:
test_results = {}

test_results['age_model'] = age_model.evaluate(
    test_features['age'],
    test_labels, verbose=0)

This is a regression with a single variable--way too simplistic for our purposes! On to bigger and better things!

## **3.2 REGRESSION: Using Multiple Variables to Predict incomeUSD in a Simple Neural Network**

You can use an almost identical setup to make predictions based on multiple inputs. This model still does the same y - mx + b except that m is a matrix and b is a vector.

This time, we use the Normalization layer that was adapted to the whole dataset.

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

When you call the model, its weight matrices will be built. You can see that the kernel (the m in y = mx + b) has a shape of (8,1). The shape (8, 1) suggests that the dense layer has 8 input features and 1 output unit. Each input feature has a corresponding weight, and there is one set of these weights for the single output unit.

In [ ]:
linear_model.layers[1].kernel

Now, we configure the model's runtime execution with the same compile and fit calls as for the single input age model:

In [ ]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_features, train_labels,
    epochs=100,
    # 100 epochs, so we turn on logging
    verbose=1,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Just like before, we want to be sure the datatype is float32 and collect the results in the test_results variable again:

In [ ]:
test_features = test_features.astype(np.float32) # Ensuring that the datatype is float
test_labels = test_labels.astype(np.float32)

test_results['linear_model'] = linear_model.evaluate(
      test_features, test_labels, verbose=0)

## Your Turn
What is the command to display the current contents of the test_results variable? Type it below and inspect the results! What do you see?

## **3.3 REGRESSION: Deep Neural Network (DNN)**


The previous section implemented linear models for single and multiple inputs in a **Simple Neural Network.**



This section implements single-input and multiple-input DNN models. The code is basically the same except **the model is expanded to include some "hidden" non-linear layers**. The name "hidden" here just means not directly connected to the inputs or outputs.

These models will contain a few more layers than the linear model:

1. The normalization layer (imagine this as hidden layer 1 in the graphic below)
2. Two hidden, nonlinear, Dense layers using the relu nonlinearity (hidden layers 2 and 3 in the graphic below)
3. A linear single-output layer because we are calculating ONE regression output.

<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/deepnn_regression.png" width="600">
</div>

Both will use the same training procedure so the compile method is included in the build_and_compile_model function below.

###**Deep Neural Network Model Code Below**
Here it is! NOTE that now, we are building and configuring the layers!




In [ ]:
def build_and_compile_model(norm):           # The input for build and compile is the norm
  model = keras.Sequential([                 # Here you are setting up the keras.Sequential architecture
      norm,                                  # This is the normalizer function we built before
      layers.Dense(64, activation='relu'),   # Here is the first hidden layer--64 nodes, built with the relu function
      layers.Dense(64, activation='relu'),   # Here is the second hidden layer, also built with the relu function
      layers.Dense(1)                        # Here is the Dense layer for a single output because we are working on a REGRESSION
  ])                                         # AND WE'RE DONE WITH BUILDING THE MODEL!

  model.compile(loss='mean_absolute_error',   # Now we configure the runtime for our model; the loss function is mean absolute error--makes sense for a regression!
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

### **3.3.1 REGRESSION--One Variable**
As before, we use the age variable to predict incomeUSD

In [ ]:
dnn_age_model = build_and_compile_model(age_normalizer)

This model has quite a few more trainable parameters than the linear models.

In [ ]:
dnn_age_model.summary()

Now, we train the model with again 100 epochs:

In [ ]:
%%time
history = dnn_age_model.fit(
    train_features['age'], train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

In [ ]:
train_labels

And we plot the outcomes:

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = dnn_age_model.predict(x)

plot_age(x, y)

Wow! Our model does a better job! But is it a good job? Not yet.

And we collect the test results:

In [ ]:
test_results['dnn_age_model'] = dnn_age_model.evaluate(
    test_features['age'], test_labels,
    verbose=0)

### **3.3.2 REGRESSION: Multiple Variables**
If you repeat this process using all the inputs it slightly improves the performance on the validation dataset.

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

Let's see if our predictions come closer to the actual data distribution (run the code below to see the graph):

In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=1, epochs=0)

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [incomeUSD]')
plt.ylabel('Predictions [incomeUSD]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

Now we can predict our test values

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

As before, we collect the results on the test set in our test_results variable:

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

# **A4. Comparing all 4 models**
Now that all the models are trained check the test-set performance and see how they did:

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [incomeUSD]']).T

## Your Turn

Which model performs the best? If you think about how each of the 4 models was built, why does your chosen model perform best?

#**B2. Classification**
In the previous section, you have learned how a Deep Learning Network works and how to build one that will predict one numeric value.

**A Deep Neural Network--Regression**
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/deepnn_regression.png" width="600">
</div>

In this module, we will start with a Deep Learning NN for Classification purposes. Instead of producing ONE numeric value, we will be configuring it to yield categorical output. This means that you'll see as many nodes in the output layer as there are level to your target categories, like the two below.

**A Deep Neural Network--Classification**
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/deepnn_classification2layers.png" width="600">
</div>

At the end of this section, you will be able to:

* Configure a deep learning Classification Network
* Distinguish the activation functions in the output layer for classifications from those for regression
* Apply regular data classification techniques to image classification
* Describe the special cases of Convolutional Neural Networks
* Solve a simple classification problem

To get started, please watch this great video that shows you where we are going with this (and if you don't remember the content from the instructor video any more, please [review that](https://youtu.be/RkiTL_T8VsY), as well):


In [3]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://www.youtube.com/embed/vF21cC-8G1U", width=560, height=315)

###**The Problem**
In the previous section, we used a regression Deep Learning network to predict incomeUSD and age. We will start this section with predicting a categorical variable in our adult dataset: Race.

You will see that setting up a Deep Learning model for a simple classification is very similar to building a regression model. In fact, the only difference, really, is in the output layer. For preprocessing, there is no difference from what we did before; in fact, in the previous model, we already prepared the "race" variable for the output layer by one-hot encoding it. Below is a summary of the code with comments

In [ ]:
#Reading in the data as a fresh adult dataframe
adult = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/adult.data.simplified25.csv")
adult.head()

# Downsizing the Dataset to just the numeric attributes
adult_dl = pd.DataFrame(adult, columns = ['age', 'educationyears', 'race','hoursperweek','incomeUSD'])

# Splitting into Training and Test Set
train_dataset = adult_dl.sample(frac=0.8, random_state=0)
test_dataset = adult_dl.drop(train_dataset.index)

# Splitting Features from Labels
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('race')
test_labels = test_features.pop('race')

In [ ]:
train_labels.head() # Let's see what the training labels look like now

In [ ]:
# Encoding the labels with pd.get_dummies
train_labels1 = pd.get_dummies(train_labels, columns=['race'], prefix='', prefix_sep='')
test_labels1 = pd.get_dummies(test_labels, columns=['race'], prefix='', prefix_sep='')

# Normalizing the input variables
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

In [ ]:
train_labels1.head() # Let's see what the training labels look like encoded

#**B3. Build the Keras Model**
Now, we can build the Sequential model and add layers one at a time until we are happy with our network architecture.

* To build the **input layer**, we need to define the number of input features. We use the **input_dim** argument and set it to 4 for the 4 input variables ('age', 'educationyears','hoursperweek','incomeUSD').
* The **output layer** will be our race attribute with 5 levels (Amer-Indian-Eskimo,	Asian-Pac-Islander,	Black,	Other,	White)


###**How do we know the number and architecture of layers in the middle?**

The short answer is: We don't. The longer answer is: We experiment until we get the best output the fastest. The even longer answer is: We can use various optimization strategies that can help us out somewhat. So, let's assume that trial and error has shown us that three layers is optimal. Furthermore, let's assume that we are going to build a **Dense Network**, aka a **fully connected** network structure, in which every node is connected with every node in the next layer.

To define this architecture, we use the Dense class. We will specify the number of neurons or nodes in the layer as the first argument, and set up the activation function with the activation argument.

Speaking of **activation function**, we will use the **rectified linear unit** or ReLU activation function on the first two layers and the Softmax function in the output layer (if our output were between 0 and 1, we would use the Sigmoid function).

###**Model Design**
So, our model looks like this:

* The model expects rows of data with 4 variables ('age', 'educationyears', 'hoursperweek', and 'incomeUSD' = the input_dim=4 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has five nodes and uses the Softmax activation function.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

#**B4. Compile the Keras Model**
Now that the model is defined, we can compile it. To do so, we must specify
* the **loss function** to use to evaluate a set of weights. In our case, we will use **categorical_crossentropy** because we have multiple nodes (categories) in the output layer.
* the **optimizer** searches through different weights for the network and any optional metrics we would like to collect and report during training. In our case, we will define the optimizer as the efficient stochastic gradient descent algorithm “**adam**“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems.
* Finally, because this is a classification problem, we will collect and report the **classification accuracy**, defined via the **metrics** argument.

In [ ]:
# compile the Keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#**B5. Train the Model**
Now that we have defined our model and compiled it, it is time to train the model on some data. We use the fit() function for this purpose. Training occurs over **epochs** and **each epoch is split into batches**.

* **Epoch**: One pass through all of the rows in the training dataset. The training process will perform a set number of iterations through the dataset  that we must specify using the 'epochs' argument.

* **Batch**: The number of dataset rows that are considered before the model weights are updated within each epoch. One epoch contains one or more batches, based on the defined 'batch_size' argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

###**How do we know the number of epochs and the batch size?**
Three words: Trial and error. Again. That's because we will be revising the model until we get the smallest loss function (aka the smallest error). Now, the model will always have **some** error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.


In [ ]:
# fit the Keras model on the dataset
model.fit(train_features, epochs=10, batch_size=10)

#**B6. Evaluate the Model**
We have trained our neural network and we can now evaluate the performance of the network on the test dataset. To evaluate your model on your training dataset, use the evaluate() function on your model and pass it the test data.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The evaluate() function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [ ]:
# evaluate the keras model
loss, accuracy = model.evaluate(test_features, test_labels1)
print('Accuracy: %.2f' % (accuracy*100),'%')

#Your Turn
As you can tell, steps 5 and 6 above don't work (yet!), and there is also a CRITICAL error in building the Keras model in Section B3.

This exercise tests your ability to **research** and **debug**. You already know a lot about running this same network on the same data, but as a regression problem, so NOW your job is to translate the regression problem into a classification problem for 'race'. All the building blocks you need are in this workbook.

**Here is your job:**
1. Fix the Keras model in section B3.
2. Fix the code in section B5 so that the model will run
4. Research how to use the predict() function to run the model on the test_features and test_labels1. Are the test_labels1 correctly encoded?

Use the fields below to work on your code.

# If You Get Stuck
To help you get unstuck:

```
# Task 2.1
adult_dl = pd.DataFrame(adult, columns = ['age', 'educationyears', 'race','hoursperweek','incomeUSD'])
adult_dl.head()
```



```
# Task 3.2
test_results

or

print(test_results)
```

